In [4]:
import tensorflow as tf, numpy as np, pandas

In [5]:
# Fonction d'initialisation des variables de poids
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

In [54]:
# Notre modèle sera, d'après l'article, sur une architecture 40 - 41 - 41 - 4
# <inputs> - 40
# <hidden> - 41
# <hidden> - 41
# <output> -  4
def model(X, w_h1, w_h2, w_o, p_keep_input=1., p_keep_hidden=1.): 
    # =====================================================================================================
    # X    : les inputs, shape [n x 40]
    # w_h1 : les poids du premier hidden layer, shape [40 x 41]
    # w_h2 : les poids du second hidden layer, shape [41 x 41]
    # w_o  : les poids du layer d'output, shape [41 x 4]
    # vérification : [n x 40] [40 x 41] [41 x 41] [41 x 4] = [n x 4]
    # =====================================================================================================
    
    X = tf.nn.dropout(X, p_keep_input)
    # --> dropout désactive certains neuronnes avec une certaine probabilité pour empêcher l'over-fitting
    h1 = tf.nn.relu(tf.matmul(X, w_h1)) # relu ? Un type de neuronnes, mais pas tout compris.
    h1 = tf.nn.dropout(h1, p_keep_hidden)
    
    h2 = tf.nn.relu(tf.matmul(h1, w_h2))
    h2 = tf.nn.dropout(h2, p_keep_hidden)

    return tf.matmul(h2, w_o)

In [4]:
# Bon, ici, on charge les données qui vont nous servir pour les tests.
# Quatre indices : Dow, Nasdaq, Rates, SNP500
Dow = pandas.read_csv("dow.csv", sep=",")
print(Dow[:5]) # Donc le Dow ressemble à ça

         Date      Open      High       Low     Close      Volume  Adj Close
0  2011-04-01  12321.02  12454.52  12301.11  12376.72  4223740000   12376.72
1  2011-03-31  12350.84  12422.96  12277.05  12319.73  3566270000   12319.73
2  2011-03-30  12280.07  12413.43  12271.52  12350.61  3809570000   12350.61
3  2011-03-29  12193.87  12310.35  12141.65  12279.01  3482580000   12279.01
4  2011-03-28  12221.19  12272.92  12197.88  12197.88  3215170000   12197.88


In [5]:
Nasdaq = pandas.read_csv("nasdaq.csv", sep=",")
print(Nasdaq[:5])

         Date     Open     High      Low    Close      Volume  Adj Close
0  2011-04-01  2796.67  2802.63  2779.71  2789.60  2090120000    2789.60
1  2011-03-31  2774.23  2783.98  2769.52  2781.07  1896420000    2781.07
2  2011-03-30  2772.36  2779.95  2763.77  2776.79  1818410000    2776.79
3  2011-03-29  2727.83  2756.89  2720.19  2756.89  1631160000    2756.89
4  2011-03-28  2752.33  2754.63  2730.68  2730.68  1669260000    2730.68


In [6]:
SnP500 = pandas.read_csv("SP500.csv", sep=",")
print(SnP500[:5])

         Date     Open     High      Low    Close      Volume  Adj Close
0  2011-04-01  1329.48  1337.85  1328.89  1332.41  4223740000    1332.41
1  2011-03-31  1327.44  1329.77  1325.03  1325.83  3566270000    1325.83
2  2011-03-30  1321.89  1331.74  1321.89  1328.26  3809570000    1328.26
3  2011-03-29  1309.37  1319.45  1305.26  1319.44  3482580000    1319.44
4  2011-03-28  1315.45  1319.74  1310.19  1310.19  2465820000    1310.19


In [7]:
Rates = pandas.read_csv("rates.csv", sep=",")
print(Rates[:5])

      date  prime
0  1970-01   8.50
1  1970-02   8.50
2  1970-03   8.39
3  1970-04   8.00
4  1970-05   8.00


In [8]:
# On observe déjà quelques soucis au niveau de l'index Rates. Ensuite, les autres indexs sont de même formats
# mais pas de même taille / avec les mêmes dates
indexes = ("Dow", "Nasdaq", "SnP500", "Rates")
for var in indexes:
    print("{} : {}".format(var.rjust(6), eval(var).size))

   Dow : 145033
Nasdaq : 70917
SnP500 : 107877
 Rates : 988


In [54]:
# Pour gérer ça, on va créer une nouvelle table.
# L'algorithme est simple : nous allons faire une intersection des données, pour ne pas avoir de jour présent
# dans une base, mais pas dans l'autre.
# A partir de là, il suffit de parcourir la base la moins fournie et de vérifier à chaque étape si la date
# est présente dans les autres. Si oui, on rajoute l'entrée complètement renseignée

# C'est le Nasdaq qui a le moins d'entrées.
# Autre info : on garde seulement la valeur de close des indices complets
datas = pandas.DataFrame(columns=['Date','Nasdaq', 'Dow', 'S&P500', 'Rates'])

# Constructing the dates in reverse order (du coup, chronologique)
for date in Nasdaq['Date'][::-1]:
    if date in Dow['Date'].values and date in SnP500['Date'].values and date[:-3] in Rates['date'].values:
        datas.loc[len(datas)] = [
            date, 
            float(Dow.loc[Dow['Date'] == date, 'Close']),
            float(Nasdaq.loc[Nasdaq['Date'] == date, 'Close']),
            float(SnP500.loc[SnP500['Date'] == date, 'Close']),
            float(Rates.loc[Rates['date'] == date[:-3], 'prime'])
        ]

AttributeError: 'DataFrame' object has no attribute 'header'

In [58]:
print(datas.head())

         Date  Nasdaq     Dow  S&P500  Rates
0  1971-02-05  876.57  100.00   96.93   5.88
1  1971-02-08  882.12  100.84   97.45   5.88
2  1971-02-09  879.79  100.76   97.51   5.88
3  1971-02-10  881.09  100.69   97.39   5.88
4  1971-02-11  885.34  101.45   97.91   5.88


In [60]:
# Maintenant que c'est fait, on sauvegarde en csv
datas.to_csv('datas.csv', sep=',', index=False)

In [22]:
# Ici commence le traitement des données
# Selon l'article que nous essayons de reproduire, nous allons appliquer une normalisation des données dans [0, 1]

# La formule est la suivante : Index(x) = (Index(x) - Min(Index))/(Max(Index) - Min(Index))
processed = pandas.read_csv("datas.csv", sep=",")
print(processed.head())

for index in 'Nasdaq Dow S&P500 Rates'.split():
    M = max(processed.loc[:, index])
    m = min(processed.loc[:, index])
    for i in range(len(processed)):
        processed.loc[i, index] = (processed.loc[i, index] - m) / (M - m)
print(processed.head())

         Date  Nasdaq     Dow  S&P500  Rates
0  1971-02-05  876.57  100.00   96.93   5.88
1  1971-02-08  882.12  100.84   97.45   5.88
2  1971-02-09  879.79  100.76   97.51   5.88
3  1971-02-10  881.09  100.69   97.39   5.88
4  1971-02-11  885.34  101.45   97.91   5.88
         Date    Nasdaq       Dow    S&P500     Rates
0  1971-02-05  0.022004  0.009037  0.023056  0.152464
1  1971-02-08  0.022413  0.009206  0.023402  0.152464
2  1971-02-09  0.022241  0.009189  0.023442  0.152464
3  1971-02-10  0.022337  0.009175  0.023362  0.152464
4  1971-02-11  0.022650  0.009328  0.023708  0.152464


In [23]:
# On sauvegarde les données <processed>
processed.to_csv('processed.csv', sep=',', index=False)

In [24]:
# Ici on les re-charge, au cas où
processed = pandas.read_csv("processed.csv", sep=",")

In [28]:
# Il reste encore quelques efforts sur les données
# On va créer un objet sample, qui contient 11 jours.

# La première nous fournira les 40 données d'input, dans une liste, sous cette forme :
# [Dow[i:i+10] Nasdaq[i:i+10] S&P[i:i+10] Rates[i:i+10]]
class Sample(object):
    def __init__(self, i):
        self.xdates = np.array(processed.loc[i:i+9, 'Date'])
        self.x = np.concatenate((
            np.array(processed.loc[i:i+9, 'Nasdaq']),
            np.array(processed.loc[i:i+9, 'Dow']),
            np.array(processed.loc[i:i+9, 'S&P500']),
            np.array(processed.loc[i:i+9, 'Rates'])
        ))
        self.ydate = np.array(processed.loc[i+10, 'Date'])
        self.y = np.array([
            processed.loc[i+10, 'Nasdaq'],
            processed.loc[i+10, 'Dow'],
            processed.loc[i+10, 'S&P500'],
            processed.loc[i+10, 'Rates']
        ])
    
sample = Sample(4)
print(sample.x)
print(sample.y)

[ 0.02264971  0.02290657  0.0229971   0.02283592  0.0226291   0.0221507
  0.02144561  0.02152068  0.02193431  0.02240241  0.00932766  0.00944781
  0.00947584  0.00938573  0.00932165  0.00917747  0.00897322  0.00898123
  0.00916546  0.0092836   0.02370797  0.02405398  0.02420702  0.02390094
  0.02347508  0.02292946  0.02225076  0.02249696  0.02292281  0.02307585
  0.15246377  0.15246377  0.15246377  0.15246377  0.15246377  0.15246377
  0.15246377  0.15246377  0.15246377  0.15246377]
[ 0.02217057  0.00930563  0.02293612  0.15246377]


In [42]:
p_train = 0.7
p_test = 1 - p_train
n = len(processed.Date.values)//11
samples = [
    Sample(i*11) for i in range(n)
]
from random import sample
indices = sample(samples, int(p_train*n))
train = np.array(indices)
test = []
for i in range(n):
    if samples[i] not in train:
        test.append(samples[i])

test = np.array(test)

print(len(train), len(test))

trX = np.array([tr.x for tr in train])
trY = np.array([tr.y for tr in train])
teX = np.array([te.x for te in test])
teY = np.array([te.y for te in test])

642 276


In [55]:
# Rappel :
# =====================================================================================================
# X    : les inputs, shape [n x 40]
# w_h1 : les poids du premier hidden layer, shape [40 x 41]
# w_h2 : les poids du second hidden layer, shape [41 x 41]
# w_o  : les poids du layer d'output, shape [41 x 4]
# vérification : [n x 40] [40 x 41] [41 x 41] [41 x 4] = [n x 4]
# =====================================================================================================

X = tf.placeholder("float", [None, 40])
Y = tf.placeholder("float", [None, 4])

w_h = init_weights([40, 41])
w_h2 = init_weights([41, 41])
w_o = init_weights([41, 4])

p_keep_input = tf.placeholder("float")   # La probabilité qui peut changer
p_keep_hidden = tf.placeholder("float")  # La probabilité qui peut changer
py_x = model(X, w_h, w_h2, w_o, p_keep_input, p_keep_hidden) # On construit le modèle avec X l'input externe

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=Y))
train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost) # Ligne importante, tout se joue ici
# On train avec un optimizer ...

In [56]:
sess = tf.Session()

# Init variables
tf.global_variables_initializer().run(session=sess)

In [57]:
for i in range(200000):
    for start, end in zip(range(0, len(trX), 10), range(10, len(trX)+1, 10)):
        # batches of 10
        sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end],
                                          p_keep_input: 1.0, p_keep_hidden: 1.0})
    if i % 200 == 0:
        print(i, teY, sess.run(py_x, feed_dict={X: teX, Y: teY, p_keep_input: 1.0, p_keep_hidden: 1.0}))

0 [[ 0.02366245  0.0099204   0.02473933  0.12695652]
 [ 0.02647029  0.01110588  0.02684198  0.12811594]
 [ 0.02494088  0.01107985  0.02581062  0.13043478]
 ..., 
 [ 0.81017419  0.52306583  0.79680877  0.        ]
 [ 0.83273631  0.52970413  0.81433524  0.        ]
 [ 0.85737102  0.55058423  0.84220857  0.        ]] [[ 0.16150177 -0.14601834  0.20454258  0.21621916]
 [ 0.16150151 -0.14601822  0.20454276  0.21621904]
 [ 0.16150154 -0.14601821  0.20454264  0.21621908]
 ..., 
 [ 0.16152002 -0.14602797  0.20456167  0.21624373]
 [ 0.16152096 -0.14602837  0.20456254  0.21624465]
 [ 0.16152124 -0.14602858  0.20456275  0.21624541]]
200 [[ 0.02366245  0.0099204   0.02473933  0.12695652]
 [ 0.02647029  0.01110588  0.02684198  0.12811594]
 [ 0.02494088  0.01107985  0.02581062  0.13043478]
 ..., 
 [ 0.81017419  0.52306583  0.79680877  0.        ]
 [ 0.83273631  0.52970413  0.81433524  0.        ]
 [ 0.85737102  0.55058423  0.84220857  0.        ]] [[ 165.3560791   164.94799805  165.40142822  165.205

KeyboardInterrupt: 

In [53]:
print(teY[0:5], '\n', sess.run(py_x, feed_dict={X: teX[0:5], Y: teY[0:5], p_keep_input: 1.0, p_keep_hidden: 1.0}))

[[ 0.02366245  0.0099204   0.02473933  0.12695652]
 [ 0.02647029  0.01110588  0.02684198  0.12811594]
 [ 0.02494088  0.01107985  0.02581062  0.13043478]
 [ 0.02049175  0.01012666  0.0222308   0.15942029]
 [ 0.02185777  0.0109597   0.02362812  0.15362319]] 
 [[  1.24919533e+12   1.24919402e+12   1.24919428e+12   1.24919703e+12]
 [  2.14952929e+12   2.14952758e+12   2.14952837e+12   2.14953008e+12]
 [  1.75960805e+12   1.75960634e+12   1.75960726e+12   1.75960923e+12]
 [  6.18552689e+11   6.18551771e+11   6.18551247e+11   6.18555703e+11]
 [  1.19751652e+12   1.19751508e+12   1.19751508e+12   1.19751875e+12]]
